In [ ]:
%load_ext autoreload
%autoreload 2
from scipy.io import loadmat
import matplotlib.pyplot as plt
import VAE
import numpy as np

## Load in the Frey Face Data

In [ ]:
ff = loadmat('../data/frey_rawface.mat')
ff = ff["ff"].T.reshape((-1, 28, 20))
print ff.shape
x_train = ff[:1800]
x_test = ff[1800:1900]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train_re = x_train.reshape((len(x_train), 28*20))
x_test_re = x_test.reshape((len(x_test), 28*20))

In [ ]:
%matplotlib notebook
plt.imshow(ff[66,:,:])

In [ ]:
v = VAE.VAE(param_file='../params/homo_gauss_gauss_freyface.json') # const var
print v.params
v.construct()

## Optionally load a previously trained VAE

In [ ]:
LOAD_MODEL = True
if LOAD_MODEL :
    v.model.load_weights('../saved_models/homo_gauss_gauss_freyface.h5')

## Fit the VAE

In [ ]:
v.fit(x_train_re, shuffle=True, add_axis=False)

## Optionally save the VAE

In [ ]:
v.model.save_weights('../saved_models/homo_gauss_gauss_freyface.h5')

## Run Model on test data

In [ ]:
pred = v.model.predict(x_test_re, batch_size=100).reshape(len(x_test_re),28,20)
print pred.shape

In [ ]:
%matplotlib notebook
example = 0
plt.imshow(pred[example,:,:])

## Encode some data

In [ ]:
z = v.encode(x_test_re, 'mu')
print z.shape

In [ ]:
%matplotlib notebook
plt.scatter(z[:,0],z[:,1])
plt.grid()
plt.show()
plt.xlabel('mu_1')
plt.ylabel('mu_2')

## Decode some data

In [ ]:
batch_size = 5
Z = np.array([[-0.15,-0.1],
              [0.15,-0.1],
              [0.15,0.08],
              [-0.15,0.08],
              [0.0,0.0]])
Z = np.array([[-10,-10],
              [10,-10],
              [10,10],
              [-10,10],
              [0.0,0.0]])
x = v.decode(Z, 'Decode1').reshape(batch_size, 28, 20)
x_0_0 = x[4,:,:]
print x.shape

## Plot Decoded Data

In [ ]:
%matplotlib notebook
for i in np.arange(0,x.shape[0]-1) :
    plt.subplot(2,2,i+1)
    plt.imshow(x[i,:,:])
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.gca().axes.get_yaxis().set_visible(False)
plt.show()

## Plot Diffs of decoded data

In [ ]:
%matplotlib notebook
for i in np.arange(0,x.shape[0]-1) :
    plt.subplot(2,2,i+1)
    plt.imshow(np.abs(x[i,:,:]-x_0_0))
    #plt.clim(0,0.004)
    plt.title('[mu_1, mu_2] = ' + str(Z[i,:]))
    plt.colorbar()
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.gca().axes.get_yaxis().set_visible(False)
plt.show()

In [ ]:
from ipywidgets import FloatSlider, interact

In [ ]:
%matplotlib notebook
z1 = FloatSlider(min=-15, max=15, step=1, value=0)
z2 = FloatSlider(min=-15, max=15, step=1, value=0)

@interact(z1=z1, z2=z2)
def do_thumb(z1,z2):
    Z = np.array([[z1,z2]])
    x = v.decode(Z, 'Decode1').reshape(28, 20)
    face = x
    plt.figure(figsize=(5, 5))
    ax = plt.subplot(111)
    #ax.imshow(np.abs(face-x_0_0))
    ax.imshow(np.abs(face))
    plt.axis("off")